# Download an archive
The script below makes a data folder with a subfolder for the dataset. Dataset option are "World", "German" or "Dutch", select these by altering the ``dataset`` variable. The script wil skip files that are already present on the system. If you want to redownload the files first remove the old files manually.  
If the download fails due to connection errors, rerun the script and will will notice that he file did not decompress and try again.  
Note: The html-file-information.csv can be very large and take several hours and fail. Consider putting the code below code in a loop if you want to run it over night, if the download fails it will try again and if successfull it will say it already has the desired file.

In [1]:
# Dataset option are "World", "German" or "Dutch"
Dataset = "Dutch"

!mkdir data
!mkdir data/{Dataset}

import os.path

if Dataset == "German":
    if not os.path.exists("data/German/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainFrequencyExtraction/domain-frequency.csv.gz?access=VRQ4COI5RFEB6XTJZNQTBRLEZTTHJERL" --output data/German/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/German/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainGraphExtraction/domain-graph.csv.gz?access=JKEPGQ6MUC72JQB23IXOC4KOLGJYDSMN" --output data/German/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/German/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/css-file-information.csv.gz?access=I2WP4REJA3NOBU3TCAAL3OIGJKNXM46R" --output data/German/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/German/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/js-file-information.csv.gz?access=M3QSMFPLEHPZPWZIFSMZ6CT2OO7WYQ4M" --output data/German/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
        
    if not os.path.exists("data/German/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/html-file-information.csv.gz?access=E3BUHKL5P4Q3TD4LGXZV2AOMERYQ3GWL" --output data/German/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")
        
if Dataset == "World":
    if not os.path.exists("data/World/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainFrequencyExtraction/domain-frequency.csv.gz?access=SMSQY3G6IGKGRWVLGCWMA7DMCHBCKQ4K" --output data/World/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/World/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainGraphExtraction/domain-graph.csv.gz?sample=true&access=BZTA7LW5LUNMPGKWQMLKAUAXBV2E2AEC" --output data/World/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/World/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/css-file-information.csv.gz?access=UZ4PWVRXXWPF53BHM7TTYNK24P7YAIXQ" --output data/World/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/World/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/js-file-information.csv.gz?access=54X7IV7QOOAJWGHKPRGDI7HIR6W6GKPI" --output data/World/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
    
    if not os.path.exists("data/World/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/html-file-information.csv.gz?access=JQEDT3PRXOS6OXZ7LA5EC55HQJVUASDX" --output data/World/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")

if Dataset == "Dutch":
    if not os.path.exists("data/Dutch/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/DomainFrequencyExtraction/domain-frequency.csv.gz?access=QUQTRHVNIDKXN62S4D62XFHCFI7VDSH7" --output data/Dutch/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/Dutch/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/DomainGraphExtraction/domain-graph.csv.gz?access=JUNESOMDTBYDNCCGVJI5MJHW45KHAIJZ" --output data/Dutch/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/Dutch/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/css-file-information.csv.gz?access=IGBKSGXJBL3L4IJ2LXS5NPJTOY77JCT5" --output data/Dutch/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/Dutch/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/js-file-information.csv.gz?access=RPZKGWUTWJBIRKOKKHGECB7W4OHUATMO" --output data/Dutch/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
    
    if not os.path.exists("data/Dutch/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/html-file-information.csv.gz?access=52ZOSNQQMFWKW42WMQPNMGKYLEVUQHUC" --output data/Dutch/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")

unzip = "find data/" +Dataset +" -name '*.gz' -exec gunzip {} \;"
!{unzip}

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/Dutch’: File exists
domain-frequency.csv already exists
domain-frequency.csv already exists
css-file-information.csv already exists
js-file-information.csv already exists
html-file-information.csv already exists


# General helper functions

## Apply patch to fix pandas memory leak issue

The code bellow comes from https://github.com/pandas-dev/pandas/issues/2659#issuecomment-415177442  
It solves a memory leak issue caused by pandas using a library that does not release memory correctly causing memory to fill up over time and eventually crash if RAM is exceded.

This may not be a problem for smaller datasets

In [2]:
# monkeypatches.py

# Solving memory leak problem in pandas
# https://github.com/pandas-dev/pandas/issues/2659#issuecomment-12021083
import sys
import pandas as pd
from ctypes import cdll, CDLL
try:
    cdll.LoadLibrary("libc.so.6")
    libc = CDLL("libc.so.6")
    libc.malloc_trim(0)
except (OSError, AttributeError):
    libc = None

__old_del = getattr(pd.DataFrame, '__del__', None)

def __new_del(self):
    if __old_del:
        __old_del(self)
    libc.malloc_trim(0)

if libc:
    print('Applying monkeypatch for pd.DataFrame.__del__', file=sys.stderr)
    pd.DataFrame.__del__ = __new_del
else:
    print('Skipping monkeypatch for pd.DataFrame.__del__: libc or malloc_trim() not found', file=sys.stderr)

Applying monkeypatch for pd.DataFrame.__del__


## Convert a full url to a domain

In [3]:
from urllib import parse

def to_domain(full_url):
    netloc = parse.urlparse(full_url).netloc
    if netloc.startswith('www.'):
        netloc = netloc[4:]
    return netloc

## Filter dataframe with regex

The **chunksize** variable determines how much memory is used. A larger chunksize allows for faster computation, but will crash when RAM is full. If the computation crashes try restarting the kernel and/or a lower chunksize. A chunksize of 1000 should be fine for most machines. On our machine memory is not an issue, but still it is something to consider when running the same code on different machines.

In [4]:
import gc # For freeing up RAM
#import pandas as pd

def filter_df_content_regex(file, regex, chunksize):
    list_regex = [] # Appending to list before conferting to dataframe because it is computationally cheaper
    
    # Loops through the CSV in chunks and saves those that match the regex in the content column 
    for chunk in pd.read_csv(file, iterator=True, chunksize=chunksize):
        # Add all rows with the regex in the content column to list_regex
        list_regex.append(chunk[chunk.content.str.contains(regex, regex= True, na=False)])
        # chunk_offset = chunk_offset + chunksize
        del chunk
        gc.collect()
    filtered_df = pd.concat(list_regex, ignore_index=True) # Convert the list to a pandas dataframe

    return filtered_df

## Add an Internet archive Wayback Machine link to a dataframe

Use the ``crawl_date`` and ``url`` columns to estimate the Internet Archive link. The CSV only provides the date of the crawl, not the time, so the resulting link may be for the wrong capure if more than one capture is available on that date. 

In [5]:
def add_ia_url(df):
    IA_url = []
    for index, row in df.iterrows():
        url = "https://web.archive.org/web/" + str(row['crawl_date']) + "/" + row["url"]
        IA_url.append(url)
    disqus_df['IA_url'] = IA_url

## Display a specific comment

In [6]:
def display_comment(dataframe, index):
    print(dataframe[dataframe['comments'].apply(lambda x: len(x)>32)]['comments'][index])

# Restrict domain list

The code below filters out unwanted domains from the dataset an creates a new CSV file

The desired domains are extracted from a column labled "URL" in a CSV file

In [7]:
domains_list = pd.read_csv("Data/German/Top News German news Websites.csv", usecols=["URL"])

domains_in_scope = domains_list["URL"].apply(to_domain) 

In [8]:
import gc # For freeing up RAM
import pandas as pd
import csv

def filter_df_domains(input_file, output_file, chunksize):
    output = open(output_file, "w")
    output.close()
    pd.read_csv(input_file, header=None, nrows=1).to_csv(output_file, mode='a', header=False)
    
    # Loops through the CSV in chunks and saves those with URLs thst match those in the domains_in_scope 
    for chunk in pd.read_csv(input_file, iterator=True, chunksize=chunksize):
        chunk=chunk[chunk.url.apply(to_domain).isin(domains_in_scope)]
        chunk.to_csv(output_file, mode='a', header=False)
        del chunk
        gc.collect()


#filter_df_domains("Data/German/html-file-information.csv", "Data/German/inScope.csv", 1000)

# Extract wepages holding disqus.js in their HTML

The code bellow looks through a dataset for any mention of disqus.js, an indicator of the presence of the disqus commenting system. The input for ```filter_df_content_regex``` is the desired dataset file, the regular expression and the chunksize.

In [9]:
# Options are "Data/German/html-file-information.csv" "Data/World/html-file-information.csv" and "Data/Dutch/html-file-information.csv"
# or any other dataset on the system.
disqus_df = filter_df_content_regex("Data/Dutch/html-file-information.csv", "(?i)disqus\.js", 10000)

Count the pages with disqus.js in their HTML

In [10]:
len(disqus_df)

258

Show the unique domains

In [11]:
print(disqus_df['url'].apply(to_domain).unique())

['middleeasteye.net' 'thedailystar.net' 'blogs.computerworld.com'
 'thehill.com' 'upriser.com' 'oneworld.nl' 'nationalinterest.org'
 'news.sciencemag.org' 'blogs.spectator.co.uk']


Note that this finds more hits than grep. It seems the csv file is too large for grep. the same query in awk does give the expected number of results

```grep -i -n 'disqus\.js' C/Users/rjans/Desktop/ARCH/Data/disqus.csv```  
vs  
```awk -e '/disqus\.js/ {print $0}' C/Users/rjans/Desktop/ARCH/Data/html-file-information-German.csv```

Save the results in a seperate CSV (only needs to be done once)

In [12]:
#disqus_df.to_csv("disqus.csv")

Convert **content** column to single line.

In [13]:
for line in disqus_df.content:
    disqus_df.content = disqus_df.content.replace(line, " ".join(line.splitlines()))

Save as single line content in a seperate CSV (only needs to be done once)

In [14]:
#disqus_df.to_csv("disqus_single_line.csv")

# For the found pages extract the comment structure

The code below requires beautifulsoup4. use the commented out import and pip command to install the library in the right place if you do not already beautifulsoup4

In [15]:
#import sys
#!{sys.executable} -m pip install beautifulsoup4
from bs4 import BeautifulSoup

Extract the disqus comments from the div tag with ```id="disqus_thread"```

In [16]:
comments = []
for line in disqus_df.content:
    soup = BeautifulSoup(line, 'html.parser')
    try:
        comments.append(soup.find('div', attrs={'id': 'disqus_thread'}).prettify()) # .prettify() makes the HTML of the comments more human readable
    except AttributeError:
        comments.append("")
        
disqus_df['comments'] = comments

In [17]:
#pd.set_option('display.max_colwidth', 500) 
disqus_df[['crawl_date','url','comments']]

,crawl_date,url,comments
0,20150402,http://www.middleeasteye.net/essays/foreign-ac...,"<div id=""disqus_thread"">\n <noscript>\n <p>\n..."
1,20160407,http://www.thedailystar.net/frontpage/friend-n...,"<div id=""disqus_thread"">\n <noscript>\n <p>\n..."
2,20120209,http://blogs.computerworld.com/19664/facebook_...,"<div id=""disqus_thread"">\n</div>\n"
3,20170723,http://thehill.com/homenews/administration/342...,"<div id=""disqus_thread"">\n <noscript>\n <p>\n..."
4,20210303,http://upriser.com/posts/alarm-bells-toll-for-...,"<div id=""disqus_thread"">\n</div>\n"
...,...,...,...
253,20160903,https://www.oneworld.nl/woesteland-zomerweken-...,"<div id=""disqus_thread"">\n <noscript>\n <p>\n..."
254,20160404,https://www.oneworld.nl/zazu-duurzame-sociale-...,"<div id=""disqus_thread"">\n <noscript>\n <p>\n..."
255,20160906,https://www.oneworld.nl/zazu-duurzame-sociale-...,"<div id=""disqus_thread"">\n <noscript>\n <p>\n..."
256,20150320,http://www.oneworld.nl/zware-gevechten-zuid-sudan,"<div id=""disqus_thread"">\n <noscript>\n <p>\n..."


# Add an internet archive URL to the dataframe

In [18]:
add_ia_url(disqus_df)

Filter out comment fields that are so short that they do net contain comments (based on the length).  
32 characters is currently the shortest 

In [19]:
#pd.set_option('display.max_colwidth', 50)
disqus_df[disqus_df['comments'].apply(lambda x: len(x)>32)][['comments','IA_url']]

,comments,IA_url
0,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20150402/http://ww...
1,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20160407/http://ww...
3,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20170723/http://th...
5,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20170406/https://w...
14,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20160301/https://w...
...,...,...
253,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20160903/https://w...
254,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20160404/https://w...
255,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20160906/https://w...
256,"<div id=""disqus_thread"">\n <noscript>\n <p>\n...",https://web.archive.org/web/20150320/http://ww...


Display a specific comment

In [20]:
display_comment(disqus_df, 257)

<div id="disqus_thread">
 <noscript>
  <p>
   <a href="http://oneworldnl.disqus.com/?url=http%3A%2F%2Fwww.oneworld.nl%2Fzware-gevechten-zuid-sudan">
    View the discussion thread.
   </a>
  </p>
 </noscript>
</div>

